In [88]:
# -*- coding: utf-8 -*-

import os
import json
import csv
import intervaltree
from datetime import datetime,timedelta

videoDurations = []       
with open('/Users/davidlemay/Google Drive/MOOC Time Series Analysis/Bucket 1/TeachersCollegeX-BDE1x-2T2015-course_structure-prod-analytics.json', 'r') as f:
    videoDurations = json.load(f)
    
for root, dirs, files in os.walk('/Users/davidlemay/Google Drive/MOOC Time Series Analysis/Code'):
    for name in files:        
        if '.DS_Store' in name:
            continue
        
        path = os.path.join(root, name)
        data = []
        with open(path, 'r') as f:
            data = json.load(f)  
            
        activity = sorted(data, key=lambda k: k['time']) 
        video = json.loads(activity[0]['event']).get('code')
        user = activity[0]['context'].get('user_id')
        
        for item in videoDurations.items():
            if item[1]['metadata'].get('youtube_id_1_0') == video:
                videoDuration = int(item[1]['metadata'].get('duration'))             
                loads = []
                plays = []
                playTimes = []
                pauseTimes = []
                pauses = []
                rewinds = []
                fforwards = []
                pbrChange = []

                for log in activity:
                    event = log['event_type']
                    eventTime = datetime.strptime(log['time'],'%Y-%m-%dT%H:%M:%S.%f+00:00') #timestamp ex: "2015-08-24T19:40:41.217890+00:00"
                    if activity.index(log) + 1 < len(activity):
                        nextEvent = activity[(activity.index(log)+1)]['event_type']
                        nextEventTime = activity[(activity.index(log)+1)]['time'] #timestamp ex: "2015-08-24T19:40:41.217890+00:00"
                        nextEventTime = datetime.strptime(nextEventTime, '%Y-%m-%dT%H:%M:%S.%f+00:00')
                        nextEventLog = json.loads(activity[(activity.index(log)+1)]['event'])
                        if event == 'load_video':
                            loads.append(eventTime) #compare timestamp for repeated loading         
                        elif event == 'play_video':
                            old_time = json.loads(log['event']).get('currentTime') #in seconds
                            new_time = json.loads(log['event']).get('currentTime')
                            if nextEvent == 'seek_video':
                                new_time = nextEventLog['old_time'] #in seconds
                            elif nextEvent == 'pause_video' or nextEvent == 'stop_video':
                                new_time = nextEventLog['currentTime'] #in seconds
                            elif nextEvent == 'speed_change_video':
                                new_time = nextEventLog['current_time'] #in seconds
                            if new_time > old_time:
                                plays.append((old_time, new_time))
                            playTimes.append(nextEventTime-eventTime)
                        elif event == 'pause_video':
                            pauseTimes.append(nextEventTime-eventTime)
                            pauses.append(log['time'])
                        elif event == 'seek_video':
                            old_time = json.loads(log['event']).get('old_time') #in seconds
                            new_time = json.loads(log['event']).get('new_time') #in seconds
                            if new_time < old_time:
                                rewinds.append((new_time, old_time))
                            elif new_time > old_time:
                                fforwards.append((new_time, old_time))
                        elif event == 'speed_change_video':
                            pbrChange.append(float(json.loads(log['event']).get('new_speed')))

                avgPBR=0
                stdPBR=0
                tree = IntervalTree.from_tuples(plays)
                if tree.merge_overlaps() is not None:
                    tree.merge_overlaps()
                totalPlayed = []
                for interval in tree:
                    totalPlayed.append(interval[1]-interval[0])
                fracSpent = sum([x.total_seconds() for x in playTimes])/videoDuration #real time relative to duration
                fracComp = sum(totalPlayed)/videoDuration#percentage played, not counting repeats
                fracPlay = sum([(y-x) for (x,y) in plays])/videoDuration #amount played, with repetition
                numPauses = len(pauses)
                fracPaused = sum([x.total_seconds() for x in pauseTimes])/videoDuration #real time relative to duration
                if len(pbrChange) is not 0:
                    avgPBR = sum(pbrChange)/len(pbrChange)
                    stdPBR = sum([(x - avgPBR)**2 for x in pbrChange])/len(pbrChange)
                numRWs = len(rewinds)
                numFFs = len(fforwards)

                with open('/Users/davidlemay/Google Drive/MOOC Time Series Analysis/user-performance.csv', 'a') as f:
                    writer = csv.writer(f)
                    row = user,video,fracSpent,fracComp,fracPlay,numPauses,fracPaused,avgPBR,stdPBR,numRWs,numFFs
                    writer.writerow(row)

In [83]:
print(json.dumps(videoDurations, indent=4,sort_keys=True))

{
    "block-v1:TeachersCollegeX+BDE1x+2T2015+type@chapter+block@22b0e3999f8e4d12a43b8b21bfa0eaa3": {
        "category": "chapter",
        "children": [
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@3db97c138c3549e7a035db4a6f08ee8a",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@d869d696539a4595a2463442c95c19cb",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@c7a795af2c8d4a5fab8d8f860a891886",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@dbb16fa4be4948cbb2705251332d0072",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@01f2a722ae454f32adf77b0fa5fc1091",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@6aaee559b4954fd9ba2b9914a345f942"
        ],
        "metadata": {
            "display_name": "Week 1: Prediction",
            "start": "2015-07-01T17:00:00Z"
        }
    },
    "block-v1:TeachersCollegeX+BDE1x+2T2015+type

In [75]:
for item in videoDurations.items():
    if item[1]['metadata'].get('youtube_id_1_0') is not None:
        print(item[1]['metadata'].get('youtube_id_1_0') + ': ' + item[1]['metadata'].get('duration'))


RuFaWB3AQCs: 636
wFIfJRce3OA: 637
aQmWf1YThFI: 545
mvn3P3EGMlc: 506
Pao5GSohrK8: 803
1lnA73Z58Y8: 551
9NKBm52zyng: 502
3tEr0J-_qiA: 588
K4aBq-apeqM: 1126
dc5Nx3tyR8g: 416
DC70-ugtFc8: 545
WI1AVcpCYgk: 241
YedABgZX5QM: 400
QRVOW8W-xeI: 495
YkhtwMZ1k_c: 453
4RpjufPG29o: 714
O5rKkvLRXw4: 417
3GDwkIh2L7Y: 381
YTMwhK705QA: 1182
0332H_1rGTg: 276
L0Ceb5BstGo: 398
4_Qxu1wcKK4: 551
Xrjfwsg7amE: 641
maKo7vQBdbo: 437
ohyD1nHEy8g: 381
HxtJyr8eqV4: 424
RHvxDieJlfE: 36
bzvBTU1bWys: 306
fbFp0uVOtng: 460
saevAbGvRBM: 454
7r3hfJW1gz0: 518
NZ08B1Khg2c: 728
gRsD8Ibzy7Q: 637
Vxi2osKtqos: 1114
3tt2sn02iUc: 628
Mr17Z0nZUQc: 504
TOmN4kfJrso: 624
rW-VlIqczJU: 1048
PaqSY7KduX4: 495
fSzOdbO0Ba8: 279
k9Z4ibzH-1s: 636
VqGTMpyD0Io: 640
1P34cxpEdKA: 418
mot1j8D0UJo: 410
nP-ntMUTOV0: 390
U_iMCMUZRI8: 410
bOhLcsQYJ6E: 182
fGMFYTHhcHg: 517
E91VInhFTFM: 718
B9dvJYwBfmk: 322
9PDwRdyb6Sw: 723
LnZGQh01Fco: 405
oTiixxmh9-Q: 406
7r3hfJW1gz0: 519
s1eDyN68bf4: 1208
7yMsobhXzfM: 529
oFSV6-opnws: 527
I-f-SEfIntI: 602
a4ngi_xlqT

In [77]:
for item in videoDurations.items():
    if item[1]['metadata'].get('youtube_id_1_0') == video:
        videoDuration = item[1]['metadata'].get('duration')

In [79]:
print(video)

YTMwhK705QA


In [76]:
print(json.dumps(videoDurations, indent=4, sort_keys=True))

{
    "block-v1:TeachersCollegeX+BDE1x+2T2015+type@chapter+block@22b0e3999f8e4d12a43b8b21bfa0eaa3": {
        "category": "chapter",
        "children": [
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@3db97c138c3549e7a035db4a6f08ee8a",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@d869d696539a4595a2463442c95c19cb",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@c7a795af2c8d4a5fab8d8f860a891886",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@dbb16fa4be4948cbb2705251332d0072",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@01f2a722ae454f32adf77b0fa5fc1091",
            "block-v1:TeachersCollegeX+BDE1x+2T2015+type@sequential+block@6aaee559b4954fd9ba2b9914a345f942"
        ],
        "metadata": {
            "display_name": "Week 1: Prediction",
            "start": "2015-07-01T17:00:00Z"
        }
    },
    "block-v1:TeachersCollegeX+BDE1x+2T2015+type

In [98]:
print(sum([x.total_seconds() for x in pauseTimes]))

598.0762820000002


In [51]:
# -*- coding: utf-8 -*-

import os
import json
import csv
from intervaltree import Interval, IntervalTree
from datetime import datetime,timedelta


path = '/Users/davidlemay/Google Drive/MOOC Time Series Analysis/Code/-vf-3c1ab43459444a6fa1ff68f15843e2c3.json'
data = []
with open(path, 'r') as f:
    data = json.load(f)       
activity = sorted(data, key=lambda k: k['time']) 

videoDurations = []       
with open('/Users/davidlemay/Google Drive/MOOC Time Series Analysis/Bucket 1/TeachersCollegeX-BDE1x-2T2015-course_structure-prod-analytics.json', 'r') as f:
    videoDurations = json.load(f)
video = json.loads(activity[0]['event']).get('code')
user = activity[0]['context'].get('user_id')
videoDuration = 0
for item in videoDurations.items():
    if item[1]['metadata'].get('youtube_id_1_0') == video:
        videoDuration = int(item[1]['metadata'].get('duration'))
loads = []
plays = []
playTimes = []
pauseTimes = []
pauses = []
rewinds = []
fforwards = []
pbrChange = []

for log in activity:
    event = log['event_type']
    eventTime = datetime.strptime(log['time'],'%Y-%m-%dT%H:%M:%S.%f+00:00') #timestamp ex: "2015-08-24T19:40:41.217890+00:00"
    if activity.index(log) + 1 < len(activity):
        nextEvent = activity[(activity.index(log)+1)]['event_type']
        nextEventTime = activity[(activity.index(log)+1)]['time'] #timestamp ex: "2015-08-24T19:40:41.217890+00:00"
        nextEventTime = datetime.strptime(nextEventTime, '%Y-%m-%dT%H:%M:%S.%f+00:00')
        nextEventLog = json.loads(activity[(activity.index(log)+1)]['event'])
        if event == 'load_video':
            loads.append(eventTime) #compare timestamp for repeated loading         
        elif event == 'play_video':
            old_time = json.loads(log['event']).get('currentTime') #in seconds
            new_time = json.loads(log['event']).get('currentTime')
            if nextEvent == 'seek_video':
                new_time = nextEventLog['old_time'] #in seconds
            elif nextEvent == 'pause_video' or nextEvent == 'stop_video' or nextEvent == 'speed_change_video':
                new_time = nextEventLog['currentTime'] #in seconds
            plays.append((old_time, new_time))
            playTimes.append(nextEventTime-eventTime)
        elif event == 'pause_video':
            pauseTimes.append(nextEventTime-eventTime)
            pauses.append(log['time'])
        elif event == 'seek_video':
            old_time = json.loads(log['event']).get('old_time') #in seconds
            new_time = json.loads(log['event']).get('new_time') #in seconds
            if new_time < old_time:
                rewinds.append((new_time, old_time))
            elif new_time > old_time:
                fforwards.append((new_time, old_time))
        elif event == 'speed_change_video':
            pbrChange.append(json.loads(log['event']).get('new_speed'))
            
avgPBR=1
stdPBR=0
tree = IntervalTree.from_tuples(plays)
totalPlayed = []
for interval in tree:
    totalPlayed.append(interval[1]-interval[0])
fracSpent = sum([x.total_seconds() for x in playTimes])/videoDuration #real time relative to duration
fracComp = sum(totalPlayed)/videoDuration#percentage played, not counting repeats
fracPlay = sum([(y-x) for (x,y) in plays])/videoDuration #amount played, with repetition
numPauses = len(pauses)
fracPaused = sum([x.total_seconds() for x in pauseTimes])/videoDuration #real time relative to duration
if len(pbrChange) is not 0:
    avgPBR = sum(pbrChange)/len(pbrChange)
    stdPBR = sum([(x - avgPBR)**2 for x in pbrChange])/len(pbrChange)
numRWs = len(rewinds)
numFFs = len(fforwards)

print(user,video,fracSpent,fracComp,fracPlay,numPauses,fracPaused,avgPBR,stdPBR,numRWs,numFFs)

4691550 0332H_1rGTg 1.0554794855072462 1.031416518115942 1.0314165181159423 21 0.9582558079710146 1 0 1 0


In [39]:
print(plays)

[(0, 27.94607), (28.281259, 32.906032), (33.190674, 51.501818), (35.68, 71.885192), (72.227386, 75.137934), (75.408452, 92.339522), (92.683711, 93.82457), (94.216197, 118.109111), (118.619827, 124.522664), (124.623546, 126.625431), (126.932388, 136.845199), (137.102497, 140.477431), (140.910485, 143.804794), (144.178322, 149.433122), (149.798421, 151.539879), (151.661191, 178.484627), (178.647163, 186.667869), (186.931402, 194.861345), (195.406506, 210.826316), (211.242159, 230.654719), (231.048341, 241.974578), (242.170449, 253.076463), (253.338999, 275.525648)]


In [55]:
print(IntervalTree.from_tuples(plays))     

IntervalTree([Interval(0, 27.94607), Interval(28.281259, 32.906032), Interval(33.190674, 51.501818), Interval(35.68, 71.885192), Interval(72.227386, 75.137934), Interval(75.408452, 92.339522), Interval(92.683711, 93.82457), Interval(94.216197, 118.109111), Interval(118.619827, 124.522664), Interval(124.623546, 126.625431), Interval(126.932388, 136.845199), Interval(137.102497, 140.477431), Interval(140.910485, 143.804794), Interval(144.178322, 149.433122), Interval(149.798421, 151.539879), Interval(151.661191, 178.484627), Interval(178.647163, 186.667869), Interval(186.931402, 194.861345), Interval(195.406506, 210.826316), Interval(211.242159, 230.654719), Interval(231.048341, 241.974578), Interval(242.170449, 253.076463), Interval(253.338999, 275.525648)])


In [49]:
tree = IntervalTree.from_tuples(plays)
totalPlayed = []
for interval in tree:
    totalPlayed.append(interval[1]-interval[0])
print(sum(totalPlayed)/int(videoDuration))

1.031416518115942


In [6]:
# -*- coding: utf-8 -*-
import json
data = []
path = '-vf-0158b22af5944ea88841e48951f1faad.json'
with open(path) as f:
    data = json.load(f)

In [19]:
print(json.dumps(activity, indent=4, sort_keys=True))

[
    {
        "accept_language": "en-US,en;q=0.8,ru;q=0.6,uk;q=0.4,de;q=0.2",
        "agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.89 Safari/537.36",
        "context": {
            "course_id": "course-v1:TeachersCollegeX+BDE1x+2T2015",
            "org_id": "TeachersCollegeX",
            "path": "/event",
            "user_id": 4691550
        },
        "event": "{\"code\": \"wFIfJRce3OA\", \"id\": \"0158b22af5944ea88841e48951f1faad\"}",
        "event_source": "browser",
        "event_type": "load_video",
        "host": "courses.edx.org",
        "ip": "93.73.246.15",
        "name": "load_video",
        "page": "https://courses.edx.org/courses/course-v1:TeachersCollegeX+BDE1x+2T2015/courseware/474d7a5fab0c464890c5d05db65310ba/d95ab68a6a09456e98b205ecbbc3681f/",
        "referer": "https://courses.edx.org/courses/course-v1:TeachersCollegeX+BDE1x+2T2015/courseware/474d7a5fab0c464890c5d05db65310ba/d95ab68a6a09456e98b205ecbbc

In [16]:
activity = sorted(data, key=lambda k: k['time'])